In [1]:
!pip install pandas


In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import joblib
from matplotlib.colors import ListedColormap

plt.style.use("fivethirtyeight")

In [61]:
class Perceptron:
    
    def __init__(self, eta:float=None,epoch:int=None):
        
        self.weights = np.random.randn(3)*1e-4 #small random weights
        training = (eta is not None) and (epoch is not None)
        if training:
            print(f"initial weights before training: \n{self.weights}")
        self.eta = eta
        self.epoch = epoch
    
    #function starts with underscore signifies internal method
    def _z_outcome(self,inputs,weights):
        return np.dot(inputs,weights)
    
    def activation_function(self,z):
        return np.where(z>0,1,0 )
    
    def fit(self,X,y):
        self.X = X
        self.y = y
        
        X_with_bias = np.c_[self.X, -np.ones((len(self.X),1))]
        print(f"X_with_bias: \n{X_with_bias}")
        
        for epoch in range(self.epoch):
            print("--"*10)
            print(f"for epoch >> {epoch}")
            print("--"*10)
            
            z = self._z_outcome(X_with_bias, self.weights)
            y_hat = self.activation_function(z)
            print(f"predicted value after forward pass: \n{y_hat}")
            
            self.error = self.y - y_hat
            print(f"error: \n{self.error}")
            
            self.weights = self.weights + self.eta * np.dot(X_with_bias.T,self.error)
            print(f"updated weights after epoch: {epoch}/{self.epoch}: \n{self.weights}")
            print("##"*10)
            
    def predict(self, X):
        X_with_bias = np.c_[X, -np.ones((len(X),1))]
        z = self._z_outcome(X_with_bias,self.weights)
        return self.activation_function(z)
    
    def total_loss(self):
        total_loss = np.sum(self.error)
        print(f"\ntotal loss: {total_loss}\n")
        return total_loss
    
    def _create_directory_return_path(self,model_dir,filename):
        os.makedirs(model_dir,exist_ok=True)
        return os.path.join(model_dir,filename)
    
    def save(self,filename,model_dir=None):
        if model_dir is not None:
            model_file_path = self._create_directory_return_path(model_dir,filename)
            joblib.dump(self,model_file_path)
        else:
            model_file_path = self._create_directory_return_path('model',filename)
            joblib.dump(self,model_file_path)

    def load(self,filepath):
        return joblib.load(filepath)


In [62]:
AND = {
    'x1':[0,0,1,1],
    'x2':[0,1,0,1],
    'y':[0,0,0,1],
}

df_and = pd.DataFrame(AND)
df_and

,x1,x2,y
0,0,0,0
1,0,1,0
2,1,0,0
3,1,1,1


In [63]:
def prepare_data(df,target_col='y'):
    X = df.drop(target_col,axis=1)
    
    y = df[target_col]
    
    return X,y

In [64]:
X,y = prepare_data(df_and)

ETA = 0.1

EPOCHS = 10

model_and = Perceptron(eta =ETA, epoch = EPOCHS)
model_and.fit(X,y)


_ = model_and.total_loss()

initial weights before training: 
[ 2.05972805e-04  1.02161533e-04 -8.10276492e-05]
X_with_bias: 
[[ 0.  0. -1.]
 [ 0.  1. -1.]
 [ 1.  0. -1.]
 [ 1.  1. -1.]]
--------------------
for epoch >> 0
--------------------
predicted value after forward pass: 
[1 1 1 1]
error: 
0   -1
1   -1
2   -1
3    0
Name: y, dtype: int64
updated weights after epoch: 0/10: 
[-0.09979403 -0.09989784  0.29991897]
####################
--------------------
for epoch >> 1
--------------------
predicted value after forward pass: 
[0 0 0 0]
error: 
0    0
1    0
2    0
3    1
Name: y, dtype: int64
updated weights after epoch: 1/10: 
[2.05972805e-04 1.02161533e-04 1.99918972e-01]
####################
--------------------
for epoch >> 2
--------------------
predicted value after forward pass: 
[0 0 0 0]
error: 
0    0
1    0
2    0
3    1
Name: y, dtype: int64
updated weights after epoch: 2/10: 
[0.10020597 0.10010216 0.09991897]
####################
--------------------
for epoch >> 3
--------------------
predict

In [65]:
model_and.predict(X=[[1,1]])

array([1])

In [67]:
model_and.save(filename='and.model')

In [68]:
#model re-load
reload_and = Perceptron().load(filepath='model/and.model')

In [69]:
reload_and.predict(X=[[1,0]])

array([0])

In [23]:
#OR gate
OR = {
    'x1':[0,0,1,1],
    'x2':[0,1,0,1],
    'y':[0,1,1,1],
}

df_or = pd.DataFrame(OR)
df_or

,x1,x2,y
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,1


In [28]:
X,y = prepare_data(df_or)

In [31]:
model_or = Perceptron(eta=0.1,epoch=10)

model_or.fit(X,y)

_ = model_and.total_loss()

initial weights before training: 
[-1.31147424e-04 -1.33248323e-04 -3.97292265e-05]
X_with_bias: 
[[ 0.  0. -1.]
 [ 0.  1. -1.]
 [ 1.  0. -1.]
 [ 1.  1. -1.]]
--------------------
for epoch >> 0
--------------------
predicted value after forward pass: 
[1 0 0 0]
error: 
0   -1
1    1
2    1
3    1
Name: y, dtype: int64
updated weights after epoch: 0/10: 
[ 0.19986885  0.19986675 -0.20003973]
####################
--------------------
for epoch >> 1
--------------------
predicted value after forward pass: 
[1 1 1 1]
error: 
0   -1
1    0
2    0
3    0
Name: y, dtype: int64
updated weights after epoch: 1/10: 
[ 0.19986885  0.19986675 -0.10003973]
####################
--------------------
for epoch >> 2
--------------------
predicted value after forward pass: 
[1 1 1 1]
error: 
0   -1
1    0
2    0
3    0
Name: y, dtype: int64
updated weights after epoch: 2/10: 
[ 1.99868853e-01  1.99866752e-01 -3.97292265e-05]
####################
--------------------
for epoch >> 3
--------------------
p

In [33]:
#XOR gate

XOR = {
    'x1':[0,0,1,1],
    'x2':[0,1,0,1],
    'y':[0,1,1,0],
}

df_xor = pd.DataFrame(XOR)

In [34]:
df_xor

,x1,x2,y
0,0,0,0
1,0,1,1
2,1,0,1
3,1,1,0


In [35]:
X,y = prepare_data(df_xor)

In [38]:
model_xor = Perceptron(eta=0.1,epoch=EPOCHS)

model_xor.fit(X,y)

_ = model_xor.total_loss()

initial weights before training: 
[-5.81782467e-05 -1.29571367e-04  1.71174484e-04]
X_with_bias: 
[[ 0.  0. -1.]
 [ 0.  1. -1.]
 [ 1.  0. -1.]
 [ 1.  1. -1.]]
--------------------
for epoch >> 0
--------------------
predicted value after forward pass: 
[0 0 0 0]
error: 
0    0
1    1
2    1
3    0
Name: y, dtype: int64
updated weights after epoch: 0/10: 
[ 0.09994182  0.09987043 -0.19982883]
####################
--------------------
for epoch >> 1
--------------------
predicted value after forward pass: 
[1 1 1 1]
error: 
0   -1
1    0
2    0
3   -1
Name: y, dtype: int64
updated weights after epoch: 1/10: 
[-5.81782467e-05 -1.29571367e-04  1.71174484e-04]
####################
--------------------
for epoch >> 2
--------------------
predicted value after forward pass: 
[0 0 0 0]
error: 
0    0
1    1
2    1
3    0
Name: y, dtype: int64
updated weights after epoch: 2/10: 
[ 0.09994182  0.09987043 -0.19982883]
####################
--------------------
for epoch >> 3
--------------------
p